In [4]:
import cvxpy as cp
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pickle

# Create a random graph
#G = nx.gnm_random_graph(10, 15)

file_path = "./graphs/satlib/CBS_k3_n100_m403_b10_340.gpickle"

with open(file_path, 'rb') as f:
    G = pickle.load(f)

G = nx.gnm_random_graph(5, 15)

#G = nx.read_gpickle('/content/CBS_k3_n100_m403_b10_18.gpickle')
n = len(G.nodes)
m = len(G.edges)
##print(G.edges)
#complement_G = nx.complement(G)

print(n,m)

# Get the adjacency matrix
A = np.array(nx.adjacency_matrix(G).todense())

# Number of vertices
n = G.number_of_nodes()

# Define the optimization variable
X = cp.Variable((n, n), symmetric=True)

# Formulate the optimization problem
objective = cp.Maximize(cp.trace(np.ones((n, n)) @ X))
constraints = [
    cp.trace(X) == 1,
    X >> 0,
]

# Add constraints for X_{i,j} = 0 for all {i, j} in E
for edge in G.edges:
    i, j = edge
    constraints.append(X[i, j] == 0)
    constraints.append(X[j, i] == 0)

# Create the CVXPY problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Extract the solution
X_solution = X.value

print(X.value)

# Display the results
print("Optimal value:", problem.value)
#print("Optimal matrix X:\n", X_solution)
print("Di")


5 10
[[2.00000000e-01 1.01769125e-10 1.01769125e-10 1.01769125e-10
  1.01769125e-10]
 [1.01769125e-10 2.00000000e-01 1.01769125e-10 1.01769125e-10
  1.01769125e-10]
 [1.01769125e-10 1.01769125e-10 2.00000000e-01 1.01769125e-10
  1.01769125e-10]
 [1.01769125e-10 1.01769125e-10 1.01769125e-10 2.00000000e-01
  1.01769125e-10]
 [1.01769125e-10 1.01769125e-10 1.01769125e-10 1.01769125e-10
  2.00000000e-01]]
Optimal value: 1.0000000014335912
Di


In [ ]:
############### SDP MIS INTERPRETATION:-

# The diagonal elements of X (the values on the main diagonal) represent the "strength" or "weight" of each vertex in being part of the independent set.
# Larger values indicate a higher likelihood of being in the independent set.
# The off-diagonal elements of X are related to the edges in the graph.
# If the value of X (i,j) is close to 0, it suggests that the vertices i and j are less likely to be in the independent set simultaneously.
# Rounding Techniques: To obtain an approximate solution for the maximum independent set, rounding techniques can be applied.
# For example, spectral rounding involves computing the eigenvector associated with the largest eigenvalue of X and then using the values of this eigenvector to determine the vertices in the independent set.

In [ ]:
print(X_solution[1,88])
print(X_solution[88,1])

In [ ]:
### zero all entries X(i,j) if (i,j) \in E
### create a dictionary of key = edge (u,v) in G' and value is equation to X(u,v)

G_hat_edges_strength_dict = {}

for v in range(n):
  for u in range(n):
    if (v,u) in G.edges():
      X_solution[u,v] = 0.0
      X_solution[v,u] = 0.0
    if (v,u) not in G.edges():
      G_hat_edges_strength_dict[(v,u)] =  X_solution[v,u]



In [ ]:
print(G_hat_edges_strength_dict[(3,0)])

In [ ]:

# Extract values as a NumPy array
values_array = np.array(list(G_hat_edges_strength_dict.values()))

# Min-Max Normalization
normalized_values_MAX = values_array / np.max(values_array)
normalized_values_MINMAX = (values_array - np.min(values_array)) / (np.max(values_array) - np.min(values_array))

# Update the dictionary with normalized values
G_hat_edges_strength_dict_normalized_MAX    = {key: normalized_values_MAX[i] for i, key in enumerate(G_hat_edges_strength_dict)}
G_hat_edges_strength_dict_normalized_MINMAX = {key: normalized_values_MINMAX[i] for i, key in enumerate(G_hat_edges_strength_dict)} ######## USE THIS


In [ ]:
print(list(G_hat_edges_strength_dict.values())[0:10])
print(list(G_hat_edges_strength_dict_normalized_MAX.values())[0:10])
print(min(list(G_hat_edges_strength_dict_normalized_MAX.values())))
print(max(list(G_hat_edges_strength_dict_normalized_MAX.values())))
print(list(G_hat_edges_strength_dict_normalized_MINMAX.values())[0:10])
print(min(list(G_hat_edges_strength_dict_normalized_MINMAX.values())))
print(max(list(G_hat_edges_strength_dict_normalized_MINMAX.values())))

print("key of min value: ", min(G_hat_edges_strength_dict_normalized_MINMAX, key=lambda k: G_hat_edges_strength_dict_normalized_MINMAX[k]))

print(G.degree(22))
print(G.degree(203))

In [ ]:
G_hat_edges_strength_dict_normalized_MINMAX

In [ ]:
#print(100*np.diag(X_solution))


sol_diag = np.diag(X_solution)

sol_diag_normalized_MAX = sol_diag / np.max(sol_diag) # Values here are between [0,1] by very near 0.9 #This can be used instead of e^T

sol_diag_normalized_MINMAX = (sol_diag - np.min(sol_diag)) / (np.max(sol_diag) - np.min(sol_diag)) # Values here varies alot between [0,1]... This can be used for initialization

print(sol_diag_normalized_MAX[0:10])

print(min(sol_diag_normalized_MAX))
print(max(sol_diag_normalized_MAX))

print(sol_diag_normalized_MINMAX[0:10])
print(min(sol_diag_normalized_MINMAX))
print(max(sol_diag_normalized_MINMAX))


In [ ]:
G.edges

In [ ]:
print(X_solution[0,6])

In [ ]:
# spectral rounding involves computing the eigenvector associated with the largest eigenvalue of X and then using the values of this eigenvector to determine the vertices in the independent set.

# Assuming X_solution is the obtained solution from SDP
eigenvalues, eigenvectors = np.linalg.eigh(X_solution)
max_ind_set_approx = np.argmax(eigenvectors[:, -1])  # Index of the maximum value in the last eigenvector
print(eigenvectors[:, -1])
print("Approximate Maximum Independent Set:", max_ind_set_approx)


In [ ]:
# Plot the graph
pos = nx.spring_layout(G)  # You can use other layout algorithms as well
nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='skyblue', node_size=800)
# Display the plot
plt.title("Graph with Optimal Matrix X")
plt.show()